# Elementary data manipulation: Split-Apply-Combine with `tidyverse`

## 0. Standard data types and structures

### 0.1 Data types
+ logical(=boolean): `TRUE`(1) 또는 `FALSE(0)`의 값을 가지는 논리값으로, 정수로도 취급됨
+ integer: $[-2^{31} , 2^{31} - 1]$ 범위를 갖는 정수
+ numeric: 일반적인 실수 전체를 의미. integer보다 범위가 넓음
+ factor: 의미를 가지는 수준(level)을 수와 매치시킨 인수
    + ordered factor: 수준이 일정한 기준을 가지고 정렬된 경우
+ character: 문자형

In [2]:
library(tidyverse)

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 2.2.1     v purrr   0.2.5
v tibble  1.4.2     v dplyr   0.7.6
v tidyr   0.8.1     v stringr 1.3.1
v readr   1.1.1     v forcats 0.3.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [6]:
class(TRUE) # 자료형을 확인하려면 class를 이용합니다.
class(2^30)
class(as.integer(3)) # 명시적으로 정수형 선언
class(as.integer(2^32)) # 명시적으로 정수형 선언: 오류
class(1.03)
class(5.67e+04) #e식 표현법은 부동소수점 표현법이라 하며 이 수는 56700과 같음
class('My name is Lora')

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

[1] "logical"

[1] "numeric"

[1] "integer"

Warning message in eval(expr, envir, enclos):
"NAs introduced by coercion to integer range"

[1] "integer"

[1] "numeric"

[1] "numeric"

[1] "character"

In [17]:
ftest <- factor(c('Green','Blue','Red'), ordered = FALSE)
ftest
ftest2 <- factor(c('Green','Blue','Red'), levels = c('Green','Blue','Red'), ordered = TRUE)
ftest2
as.integer(ftest2)
ftest2_1 <- factor(c('Green', 'Blue', 'Red'), levels = c('Green','Blue','Red'), labels = c('G','B','R'))
ftest2_1
as.integer(ftest2_1)
ftest3 <- factor(1:10)
ftest3
ftest4 <- factor(1:5, levels = c(3, 1, 5, 4, 2), ordered = TRUE) # 지정된 수준으로 정렬된 factor; 각 수준은 정수 1~5에 순서대로 배열
ftest4
as.integer(ftest4) # 결과에 주목. 숫자 1이 두 번째 수준이므로 2가,  숫자 2가 다섯 번째 수준이므로 5가, 같은 요령으로 각각 1, 4, 3순서가 정수형으로 출력됨
str(ftest4)

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

[1] Green Blue  Red  
Levels: Blue Green Red

[1] Green Blue  Red  
Levels: Green < Blue < Red

[1] 1 2 3

[1] G B R
Levels: G B R

[1] 1 2 3

[1] 1  2  3  4  5  6  7  8  9  10
Levels: 1 2 3 4 5 6 7 8 9 10

[1] 1 2 3 4 5
Levels: 3 < 1 < 5 < 4 < 2

[1] 2 5 1 4 3

 Ord.factor w/ 5 levels "3"<"1"<"5"<"4"<..: 2 5 1 4 3


### Why do we use `factor`?
- 메모리 절약: 실제로 저장되는 것은 모든 `levels`의 레이블에 대응하는 정수가 저장되기 때문입니다.
- 아래 코드 블록과 실행 결과를 참고하시면 확실하게 이해되시리라 생각합니다.

In [24]:
## Character로 관리할 때
object.size(rep(c('Green','Blue','Red'), c(1000, 250, 400)))
## Factorize하여 관리할 때
object.size(factor(rep(c('Green','Blue','Red'), c(1000, 250, 400))))
## Factorize하여 관리하되, 순서를 강제하지 않을 때
object.size(factor(rep(c('Green','Blue','Red'), c(1000, 250, 400)), levels = c('Red', 'Blue', 'Green')))
## Factorize하여 관리하되, 순서를 강제할 때
object.size(factor(rep(c('Green','Blue','Red'), c(1000, 250, 400)), levels = c('Red', 'Blue', 'Green'), ordered = TRUE))

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

13416 bytes

7232 bytes

7232 bytes

7296 bytes

## 0.2 Data classes
+ `vector`: 1차원 숫자 뭉치
+ `matrix`: 2차원 숫자 뭉치
+ __`data.frame`__: 자료를 고유한 열 이름을 갖는 틀(frame)에 담은 것으로, 자료 분석의 일반적 단위
+ `array`: 숫자 뭉치를 여러 개의 `matrix`로 저장한 형태 (3차원)
+ `list`: 모든 자료형을 저장할 수 있는 단위

In [1]:
# vectors and matrices
class(as.vector(1))
mat <- matrix(1:9, nrow = 3, ncol = 3, byrow = TRUE)
mat
class(mat)
class(as.vector(mat))
class(c(1,'1','3',3))
class(c(1,'1',factor('1')))

[1] "numeric"

1,2,3
4,5,6
7,8,9


[1] "matrix"

[1] "integer"

[1] "character"

[1] "character"

In [42]:
## data.frame 정의의 예
dat <- data.frame(field1 = c('doc', 'dog', 'dor'),
                  field2 = 1:3,
                  field3 = factor(c('alpha', 'beta', 'phi'), levels = c('phi', 'beta', 'alpha')))
dat
str(dat) # 데이터의 구조를 살펴보려면 str 함수를 이용합니다.

## data.frame 정의의 잘못된 예
dat2 <- data.frame(field1 = c('doc', 'dog', 'dor'),
                  field2 = 1:2,
                  field3 = factor(c('alpha', 'beta', 'phi'), levels = c('phi', 'beta', 'alpha')))
dat2

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

field1,field2,field3
doc,1,alpha
dog,2,beta
dor,3,phi


'data.frame':	3 obs. of  3 variables:
 $ field1: Factor w/ 3 levels "doc","dog","dor": 1 2 3
 $ field2: int  1 2 3
 $ field3: Factor w/ 3 levels "phi","beta","alpha": 3 2 1


ERROR: Error in data.frame(field1 = c("doc", "dog", "dor"), field2 = 1:2, field3 = factor(c("alpha", : arguments imply differing number of rows: 3, 2


## 1. Reading data from external sources
+ 우리가 내려받는 데이터가 항상 '곧장 사용할 수 있는' 형태로 되어 있지는 않습니다.
+ 먼저 '깨끗한' 데이터를 불러오는 방법을 알아보고, 그렇지 않은 데이터를 읽어들이는 방법을 알아봅니다.
+ 데이터를 불러온 후에는 `summary` 함수나 `str`, `head` 등의 함수를 이용해서 대략의 구조를 파악합니다.

### 1.1 Reading 'neat' data
+ `readr` 패키지의 `read_csv` 함수: 쉼표로 구분된 파일을 읽어들입니다.
+ `readxl` 패키지의 `read_excel` 함수: xlsx 형식으로 된 엑셀 파일을 읽어들입니다.
+ 이외에 `readr` 패키지의 `read_fwf` 함수는 고정폭 (fixed width) 파일을 읽어들입니다.

In [3]:
# read_csv
pla <- readr::read_csv(file = 'C:/Users/sigma/Dropbox/Chores/2018S/GeoCONDA/W1/Plastic_trade.csv',
                       col_types = 'cccddicc') # for the detail of col_types, see ?read_csv
### character for the first to the third column
### double(numeric) for the fourth to the fifth column
### integer for the sixth column
### character for the seventh to the eighth column
head(pla, 1)
dim(pla)

Flow,Reporter,Partner,Netweight_kg,Value_usd,year,Reporter_nm,Partner_nm
Import,ALB,BGR,124190,107621,2010,ALBANIA,BULGARIA


[1] 37100     8

__위에서 불러온 `pla` object는 이후에 계속 쓰입니다. __  

## 1.2 Reading 'dirty (or uncleaned)' data
- 일반적인 `data.frame` 형식으로 정리되지 않은 데이터를 불러오는 방법을 설명합니다.
- 통계청에서 내려받는 엑셀 파일들이 주로 정리되지 않은 데이터에 속합니다.
- 아래 코드 블록은 국민건강보험공단에서 내려받은 한 엑셀 데이터입니다. 위쪽에 사족이 많이 붙어 있어서 별도의 처리 없이 불러오면 `data.frame`이 뒤죽박죽이 되어 있는 모습을 볼 수 있습니다.
- `readxl::read_excel`의 `range` 인수는 `character`형의 셀 범위를 받아서 엑셀 파일을 읽어옵니다. 적절한 `range`를 입력하면 불필요한 내용들의 영향을 없앨 수 있습니다.
- 한편, `readr::read_csv`의 `skip` 인수는 `integer`을 받아서 위에서부터 `skip` 줄만큼을 빼고 데이터를 읽어옵니다.

In [ ]:
# read_excel
atc <- readxl::read_excel('C:/Users/sigma/Dropbox/Chores/2018S/GeoCONDA/W1/ATC.xlsx', range = 'A8:CW93')
str(atc)

## 2. Split-Apply-Combine

![](http://library.open.oregonstate.edu/computationalbiology/wp-content/uploads/sites/3/2016/10/III.8_50_dplyr_do.png)

## SAC: basic
+ `data.frame`을 일정한 기준으로 나누고(split), 나눈 데이터에 각각 원하는 연산을 수행하고(apply), 얻은 결과를 합치는(combine)것을 말합니다. 대용량 데이터베이스에서 자료를 분산처리하는 원리와 유사하며, `data.frame` 형태로 된 데이터를 정리하거나 집계하는 데 유용한 접근법입니다.
+ 위 그림은 어종별 중량 데이터를 종에 따라 분리하고, 평균과 표준편차를 각각 구하는 과정을 보여줍니다.
+ 이 원리 하에서, __우리는 항상 데이터를 나눌 기준에 주목__하고, 나눠진 각 데이터에 어떤 연산을 적용할 것인가를 유의해야 합니다. 
+ 아래에서는 어떤 함수가 어떠한 기능을 수행하는지 설명하겠습니다.

### All the way ahead: `%>%`
- 퍼센트 기호 두 개 사이에 '크거나 같은(>)' 기호가 들어 있는 이 표식을 __'파이프 연산자 (pipe operator)'__라 합니다.
- 최초에 데이터를 투입한 다음, 적용할 함수들을 파이프 연산자로 연결하여 처리합니다. RStudio를 사용하면 `Ctrl`+`Shift (Left)`+`M`을 눌러서 손쉽게 입력할 수 있습니다. 여러분들이 앞으로 주로 쓰게 될 Jupyter notebook에서는 이러한 기능이 지원되지 않으므로, 다소 번거롭더라도 직접 타이핑해서 처리해야 합니다.
- 파이프 연산자가 연결된 상태에서, 투입된 자료 자신을 온점 `.`으로 표현합니다. 명시적으로 처리하지 않으면 자료가 파이프 연산자 뒤 함수의 첫 번째 인자로 투입됩니다.
- 파이프 연산자의 장점으로는, 데이터 처리 과정을 순차적으로 설계하고 이해하는 데 편리하다는 점을 들 수 있습니다. 아래 예시를 보세요. 

### Grouping: `group_by`
- `dplyr::group_by`는 자료를 묶을 하위 기준을 지정하는 함수입니다.
- `group_by(groupvar)`: `groupvar`라는 필드의 고윳값 (unique values)을 기준으로 정보를 묶어서 처리하겠다는 의미입니다.
- 계획한 대로 자료를 다 묶어서 처리했으면 반드시 다음 파이프 연산자 뒤에 `ungroup` 함수를 적용해서 grouping을 해제해야 합니다. __(매우 중요!)__

### Data subsetting: `filter`
- `dplyr::filter`는 논리 조건을 부여하여 원하는 레코드만 추출하고자 할 때 사용하는 함수입니다.

### Data editing: `mutate`, `rename`
- `dplyr::mutate`는 단어 뜻 그대로 필드에 어떤 처리를 하고자 할 때 사용하는 함수입니다.
- `dplyr::rename`은 어떤 필드의 이름을 바꿀 때 사용하는 함수입니다.

### Data aggregation: `summarize`
- `dplyr::summarize`는 자료를 지정된 기준 필드의 값들에 따라서 특정한 필드를 요약하는 함수입니다. '기준 필드'를 필요로 하기 때문에 반드시 `group_by` 뒤에 와야 합니다.

In [29]:
#### Example: pipe operator
data(starwars) # Star Wars characters data
str(starwars)

Classes 'tbl_df', 'tbl' and 'data.frame':	87 obs. of  13 variables:
 $ name      : chr  "Luke Skywalker" "C-3PO" "R2-D2" "Darth Vader" ...
 $ height    : int  172 167 96 202 150 178 165 97 183 182 ...
 $ mass      : num  77 75 32 136 49 120 75 32 84 77 ...
 $ hair_color: chr  "blond" NA NA "none" ...
 $ skin_color: chr  "fair" "gold" "white, blue" "white" ...
 $ eye_color : chr  "blue" "yellow" "red" "yellow" ...
 $ birth_year: num  19 112 33 41.9 19 52 47 NA 24 57 ...
 $ gender    : chr  "male" NA NA "male" ...
 $ homeworld : chr  "Tatooine" "Tatooine" "Naboo" "Tatooine" ...
 $ species   : chr  "Human" "Droid" "Droid" "Human" ...
 $ films     :List of 87
  ..$ : chr  "Revenge of the Sith" "Return of the Jedi" "The Empire Strikes Back" "A New Hope" ...
  ..$ : chr  "Attack of the Clones" "The Phantom Menace" "Revenge of the Sith" "Return of the Jedi" ...
  ..$ : chr  "Attack of the Clones" "The Phantom Menace" "Revenge of the Sith" "Return of the Jedi" ...
  ..$ : chr  "Revenge of the 

In [52]:
## Two lines give the same results
starwars %>% .$skin_color %>% unique %>% length # self-referencing with a period (.) then pipelining 
length(unique(starwars$skin_color))

##
starwars %>% group_by(species) %>% summarize(N = n(), mean_height = mean(height), mean_mass = mean(mass)) %>% ungroup %>% head
starwars %>% group_by(species) %>% summarize(mean_height = mean(height, na.rm = TRUE), mean_mass = mean(mass, na.rm = TRUE)) %>% ungroup %>% head # See the difference to the previous line? :)
## group_by multiple fields
starwars %>% group_by(species, homeworld) %>% summarize(N = n()) %>% ungroup %>% head#


[1] 31

[1] 31

species,N,mean_height,mean_mass
Aleena,1,79,15
Besalisk,1,198,102
Cerean,1,198,82
Chagrian,1,196,NA
Clawdite,1,168,55
Droid,5,NA,NA


species,mean_height,mean_mass
Aleena,79,15.00
Besalisk,198,102.00
Cerean,198,82.00
Chagrian,196,NaN
Clawdite,168,55.00
Droid,140,69.75


species,homeworld,N
Aleena,Aleen Minor,1
Besalisk,Ojom,1
Cerean,Cerea,1
Chagrian,Champala,1
Clawdite,Zolan,1
Droid,Naboo,1


## SAC: advanced
### Selective application: `*_all`, `*_each`, `*_if`, and `*_at`
- 제목의 `*`에는 기본편에서 살펴본 `dplyr::filter`, `dplyr::mutate`, `dplyr::summarize` 등의 함수가 들어갑니다.
- 기본 집계, 변형, 추출 함수들을 복수의 필드에 한꺼번에 적용할 때, 상황에 맞는 덧붙임말을 넣어서 함수를 적용합니다.
    - `*_all`
    - `*_each`: _자료의 모든 필드에 각각_ 지정된 함수를 적용합니다.
    - `*_if`: _주어진 조건에 맞는 필드에 한해서 각각_ 지정된 함수를 적용합니다.
    - `*_at`: _지정한 필드에 한해서 각각_ 지정된 함수를 적용합니다.

In [61]:
# Compute mean height and mean mass of all species, along with excluding NA values
starwars %>% group_by(species) %>% summarize_at(.vars = vars(height, mass), .funs = funs(mean(., na.rm = TRUE))) %>% ungroup %>% head
# Count the number of unique values in character fields
starwars %>% group_by(species) %>% summarize_if(.predicate = is.character, .funs = funs(length(unique(.)))) %>% ungroup %>% head
# Find all character fields then change classes of values therein to factor, then count the number of unique values in each field
starwars %>% group_by(species) %>% mutate_if(.predicate = is.character, .funs = as.factor) %>% summarize_all(.funs = funs(length(unique(.)))) %>% ungroup %>% head

species,height,mass
Aleena,79,15.00
Besalisk,198,102.00
Cerean,198,82.00
Chagrian,196,NaN
Clawdite,168,55.00
Droid,140,69.75


species,name,hair_color,skin_color,eye_color,gender,homeworld
Aleena,1,1,1,1,1,1
Besalisk,1,1,1,1,1,1
Cerean,1,1,1,1,1,1
Chagrian,1,1,1,1,1,1
Clawdite,1,1,1,1,1,1
Droid,5,2,5,3,2,3


Warning message in mutate_impl(.data, dots):
"Unequal factor levels: coercing to character"Warning message in mutate_impl(.data, dots):
"binding character and factor vector, coercing into character vector"Warning message in mutate_impl(.data, dots):
"binding character and factor vector, coercing into character vector"Warning message in mutate_impl(.data, dots):
"binding character and factor vector, coercing into character vector"Warning message in mutate_impl(.data, dots):
"binding character and factor vector, coercing into character vector"Warning message in mutate_impl(.data, dots):
"binding character and factor vector, coercing into character vector"Warning message in mutate_impl(.data, dots):
"binding character and factor vector, coercing into character vector"Warning message in mutate_impl(.data, dots):
"binding character and factor vector, coercing into character vector"Warning message in mutate_impl(.data, dots):
"binding character and factor vector, coercing into character vect

species,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,vehicles,starships
Aleena,1,1,1,1,1,1,1,1,1,1,1,1
Besalisk,1,1,1,1,1,1,1,1,1,1,1,1
Cerean,1,1,1,1,1,1,1,1,1,1,1,1
Chagrian,1,1,1,1,1,1,1,1,1,1,1,1
Clawdite,1,1,1,1,1,1,1,1,1,1,1,1
Droid,5,5,4,2,5,3,4,2,3,5,1,1


## Data transformation
### Long/wide data frame
- 분류 속성을 어떻게 처리하는가에 따른 `data.frame`의 분류입니다.
    - long format: 분류 속성이 하나의 필드에 정리되어 세로로 긴 형태의 data frame
    - wide format: 분류 속성이 각 필드에 정리되어 가로로 긴 형태의 data frame
    - 우리는 앞으로 wide data frame을 long data frame으로 자주 바꾸게 됩니다. 시각화 파트에서 다룰 `ggplot2` 패키지가 long data frame에 친화적으로 되어 있기 때문입니다.
- `tidyr::spread`와 `tidyr::gather`는 각각 long-wide, wide-long 변환을 수행하는 함수입니다.
    - 인수 `key`는 분류 속성을 의미합니다.
    - 인수 `value`는 채워넣을 값이 있는 필드를 의미합니다.
- 아래 코드 블록은 Star Wars의 인물들을 성별과 눈동자 색에 따라 키와 몸무게의 평균이 어떠한지를 요약합니다. 분류 속성은 `gender`와 `eye_color`입니다.
- 정리된 내용을 각각 `gender`의 속성과 `eye_color`의 속성에 따라서 `height`와 `mass`를 나타내도록, 즉 wide format으로 바꾸어 보겠습니다.
- 아래 예시를 잘 활용하면 분류 속성이 여러 차원일 때, 데이터를 효과적으로 요약하여 제시할 수 있습니다.

In [33]:
starwars %>% group_by(gender, eye_color) %>% 
    summarize_at(.vars = vars(height, mass), .funs = funs(mean(., na.rm = TRUE))) %>% ungroup -> sw_summary

## gather height and mass
sw_summary %>% gather(key = vclass, value = val, 3:4) %>% # gather height and mass (the third and fourth column, respectively)
# fill values to val column, and fill the class names ('height' and 'mass') in vclass column
    mutate(agc1 = paste(gender, vclass, sep = '_'),
           agc2 = paste(eye_color, vclass, sep = '_')) -> sw_summary_wide

sw_summary_wide %>% dplyr::select(eye_color, agc1, val) %>% spread(key = agc1, value = val) %>% head(5)
sw_summary_wide %>% dplyr::select(gender, agc2, val) %>% spread(key = agc2, value = val) %>% head(5)

eye_color,female_height,female_mass,hermaphrodite_height,hermaphrodite_mass,male_height,male_mass,NA_height,NA_mass,none_height,none_mass
black,195.5,57.0,NA,NA,182.0000,79.50000,NA,NA,NaN,NaN
blue,167.0,57.8,NA,NA,189.2308,100.87500,NA,NA,NA,NA
blue-gray,NA,NA,NA,NA,182.0000,77.00000,NA,NA,NA,NA
brown,160.0,47.0,NA,NA,168.5000,69.56364,NA,NA,NA,NA
dark,NA,NA,NA,NA,NaN,NaN,NA,NA,NA,NA


gender,black_height,black_mass,blue-gray_height,blue-gray_mass,blue_height,blue_mass,brown_height,brown_mass,dark_height,...,"red, blue_height","red, blue_mass",red_height,red_mass,unknown_height,unknown_mass,white_height,white_mass,yellow_height,yellow_mass
female,195.5,57.0,NA,NA,167.0000,57.800,160.0,47.00000,NA,...,96,NaN,NA,NA,NaN,NaN,178,48,168.0000,55.00000
hermaphrodite,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
male,182.0,79.5,182,77,189.2308,100.875,168.5,69.56364,NaN,...,NA,NA,190.5,101.5,136,31.5,NA,NA,180.1111,85.71429
none,NaN,NaN,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,200.0,140.0,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,96.5,32.0,NA,NA,NA,NA,167.0000,75.00000


## 3. Real-world example: Recycling crisis

4월 초 며칠 동안, 일명 '재활용 대란'이 있었습니다. 아시다시피, 원인으로 국제 폐비닐/폐플라스틱 시세 하락으로 인한 중국으로의 수출 중단이 원인으로 지목되었습니다. 여기서, 우리는 실제 시세 하락은 얼마나 일어났는가라는 질문을 제기할 수 있습니다. 우리는 데이터를 통해서 간접적으로 그 질문을 풀어보려 합니다. 자료는 [UN Comtrade](https://comtrade.un.org/)에서 2010년에서 2017년 간의 세계 국가별 무역데이터를 내려받았고 (학내에서만 무료로 내려받을 수 있음), 미리 폐플라스틱 (HS code: 9315)에 해당하는 상품코드에 맞추어 연도별 데이터를 추출하였습니다. 과제는 단일 연도의 전체 데이터를 대상으로 수행하시게 될 것입니다.
이 데이터는 보고국(reporter)과 대상국(partner) 명칭, 수출/입 여부, 폐플라스틱 순교역량(kg), 교역액(USD) 정보, 통계연도를 담고 있습니다. 우리는 이 파일에서 몇 가지 질문에 대한 해답을 얻어내고자 합니다.

1. 지난 8년 간 세계 폐플라스틱 무역 총량을 알아봅니다.
2. 지난 8년 간 국가별 폐플라스틱 무역을 결산해봅니다.
3. 지난 8년 간 국가별 대중국 폐플라스틱 무역 총량을 알아봅니다.
4. 한국의 폐플라스틱 무역의 시계열을 액수와 중량에 따라 조사해 봅니다.
5. 지난 8년 간 세계 폐플라스틱 시세 변동을 알아봅니다.
    - 단, '세계 폐플라스틱 시세'를 데이터로부터 살펴보기 위해서 다음과 같이 조작적으로 정의합시다: '세계 폐플라스틱 시세'란 단위순중량(kg)당 교역액(USD)를 의미합니다. 폐플라스틱의 세부구분은 고려하지 않습니다.

### 3.1
- 자료에는 '세계 전체'를 아우르는 정보가 없으므로, 정보를 생성한 다음 집계하도록 합시다.
- 아래 첫 번째 코드 블록에서는 NA값을 빼고 순중량과 교역액을 합산했습니다. 이 때 제외된 값들이 무엇이었느냐에 따라서 왜곡이 발생할 수 있습니다. 두 번째 코드 블록에서 확인해 보겠습니다.
- 두 번째 코드 블록에서 `arrange`에 필드 이름 앞에 음의 기호를 붙이면 '내림차순 정렬'을 하라는 뜻입니다. 확인 결과, 순중량이 포함되지 않은 사례들이 161건 있었고, 교역액 필드를 볼 때에 교역량이 큰 사례들에서 중량이 누락되어 있으므로, 이들을 제외하고 합산한 결과에 상당한 왜곡이 있으리라고 예상할 수 있습니다. 특히 2016년 사례가 그렇습니다.
- 위의 문제점에 착안해서, 세 번째 코드 블록에서는 어느 한 필드에서도 `NA`값이 존재하지 않는 레코드들만 추출한 후 같은 분석을 수행해 봅니다.

In [15]:
str(pla)

Classes 'tbl_df', 'tbl' and 'data.frame':	37100 obs. of  8 variables:
 $ Flow        : chr  "Import" "Export" "Import" "Export" ...
 $ Reporter    : chr  "ALB" "ALB" "ALB" "ALB" ...
 $ Partner     : chr  "BGR" "CHN" "DEU" "DEU" ...
 $ Netweight_kg: num  124190 812861 2347640 678160 1516306 ...
 $ Value_usd   : num  107621 425149 943464 349723 647468 ...
 $ year        : int  2010 2010 2010 2010 2010 2010 2010 2010 2010 2010 ...
 $ Reporter_nm : chr  "ALBANIA" "ALBANIA" "ALBANIA" "ALBANIA" ...
 $ Partner_nm  : chr  "BULGARIA" "CHINA" "GERMANY" "GERMANY" ...
 - attr(*, "spec")=List of 2
  ..$ cols   :List of 8
  .. ..$ Flow        : list()
  .. .. ..- attr(*, "class")= chr  "collector_character" "collector"
  .. ..$ Reporter    : list()
  .. .. ..- attr(*, "class")= chr  "collector_character" "collector"
  .. ..$ Partner     : list()
  .. .. ..- attr(*, "class")= chr  "collector_character" "collector"
  .. ..$ Netweight_kg: list()
  .. .. ..- attr(*, "class")= chr  "collector_double" "co

In [10]:
pla %>% mutate(all = 'World') -> pla_w
pla_w %>% group_by(all) %>% 
        summarize(sum_netweight = sum(Netweight_kg, na.rm = TRUE), 
                  sum_value = sum(Value_usd, na.rm = TRUE)) %>% ungroup -> pla_wsum
pla_w %>% group_by(all, year) %>% 
        summarize(sum_netweight = sum(Netweight_kg, na.rm = TRUE), 
                  sum_value = sum(Value_usd, na.rm = TRUE)) %>% ungroup -> pla_wysum
pla_w %>% group_by(all, year, Flow) %>% 
        summarize(sum_netweight = sum(Netweight_kg, na.rm = TRUE), 
                  sum_value = sum(Value_usd, na.rm = TRUE)) %>% ungroup -> pla_wyfsum
## check
pla_wsum
pla_wysum
pla_wyfsum

all,sum_netweight,sum_value
World,237181315048,1.16271e+11


all,year,sum_netweight,sum_value
World,2010,33843255010,14901904454
World,2011,33454415677,17503964586
World,2012,33755764006,17528883383
World,2013,28969211873,16585989884
World,2014,34051406456,17146351249
World,2015,31896306484,13581726733
World,2016,26594439838,12457625514
World,2017,14616515704,6564595351


all,year,Flow,sum_netweight,sum_value
World,2010,Export,15337934825,6055398026
World,2010,Import,15788234286,8067400574
World,2010,Re-Export,2715696936,778328090
World,2010,Re-Import,1388963,777764
World,2011,Export,14822716997,7024457937
World,2011,Import,15974271606,9618216630
World,2011,Re-Export,2655678460,860032180
World,2011,Re-Import,1748614,1257839
World,2012,Export,14841538079,6983048221
World,2012,Import,15940073245,9573883559


In [9]:
pla %>% filter(is.na(Netweight_kg)) %>% arrange(-Value_usd) %>% head(10)
pla %>% filter(is.na(Netweight_kg)) %>% arrange(-Value_usd) %>% dim
pla %>% filter(is.na(Value_usd)) %>% arrange(-Netweight_kg) %>% head(10)

Flow,Reporter,Partner,Netweight_kg,Value_usd,year,Reporter_nm,Partner_nm
Export,HKG,CHN,NA,875224816,2016,HONG KONG,CHINA
Re-Export,HKG,CHN,NA,849199861,2016,HONG KONG,CHINA
Export,VNM,CHN,NA,85922017,2012,VIET NAM,CHINA
Export,GBR,IND,NA,7142041,2017,UNITED KINGDOM,INDIA
Import,VNM,THA,NA,4921349,2012,VIET NAM,THAILAND
Import,VNM,KOR,NA,4296293,2012,VIET NAM,"KOREA, REPUBLIC OF"
Import,VNM,USA,NA,3772927,2012,VIET NAM,UNITED STATES
Import,HRV,DEU,NA,2840440,2017,CROATIA,GERMANY
Import,VNM,SAU,NA,2290120,2012,VIET NAM,SAUDI ARABIA
Export,VNM,HKG,NA,1805740,2012,VIET NAM,HONG KONG


Flow,Reporter,Partner,Netweight_kg,Value_usd,year,Reporter_nm,Partner_nm


In [14]:
pla %>% mutate(all = 'World') %>% filter(!is.na(Netweight_kg) & !is.na(Value_usd)) -> pla_wc
pla_wc %>% group_by(all) %>% 
        summarize(sum_netweight = sum(Netweight_kg), 
                  sum_value = sum(Value_usd)) %>% ungroup -> pla_wcsum
pla_wc %>% group_by(all, year) %>% 
        summarize(sum_netweight = sum(Netweight_kg), 
                  sum_value = sum(Value_usd)) %>% ungroup -> pla_wcysum
pla_wc %>% group_by(all, year, Flow) %>% 
        summarize(sum_netweight = sum(Netweight_kg), 
                  sum_value = sum(Value_usd)) %>% ungroup -> pla_wcyfsum
## check
pla_wcsum
pla_wcysum
pla_wcyfsum


all,sum_netweight,sum_value
World,237181315048,114403841702


all,year,sum_netweight,sum_value
World,2010,33843255010,14901899881
World,2011,33454415677,17503963609
World,2012,33755764006,17410206260
World,2013,28969211873,16585989884
World,2014,34051406456,17146351249
World,2015,31896306484,13581705215
World,2016,26594439838,10733114974
World,2017,14616515704,6540610630


all,year,Flow,sum_netweight,sum_value
World,2010,Export,15337934825,6055398026
World,2010,Import,15788234286,8067396001
World,2010,Re-Export,2715696936,778328090
World,2010,Re-Import,1388963,777764
World,2011,Export,14822716997,7024457937
World,2011,Import,15974271606,9618215653
World,2011,Re-Export,2655678460,860032180
World,2011,Re-Import,1748614,1257839
World,2012,Export,14841538079,6891275058
World,2012,Import,15940073245,9546979599


all,year,Flow,sum_netweight,sum_value,prunit
World,2010,Export,15337934825,6055398026,394.7988
World,2010,Import,15788234286,8067396001,510.9752
World,2010,Re-Export,2715696936,778328090,286.6034
World,2010,Re-Import,1388963,777764,559.9602
World,2011,Export,14822716997,7024457937,473.8981
World,2011,Import,15974271606,9618215653,602.1067
World,2011,Re-Export,2655678460,860032180,323.8465
World,2011,Re-Import,1748614,1257839,719.3349
World,2012,Export,14841538079,6891275058,464.3235
World,2012,Import,15940073245,9546979599,598.9295


### 3.2 

In [17]:
pla_wc %>% group_by(Reporter, Reporter_nm, Flow) %>% summarize(sum_weight = sum(Netweight_kg), sum_value = sum(Value_usd)) %>% ungroup -> pla_wccf
pla_wccf %>% head(12)
pla_wccf %>% filter(Flow == 'Import') %>% arrange(-sum_value) %>% head(5)
pla_wccf %>% filter(Flow == 'Export') %>% arrange(-sum_value) %>% head(5)
pla_wccf %>% filter(Flow == 'Re-Import') %>% arrange(-sum_value) %>% head(5)
pla_wccf %>% filter(Flow == 'Re-Export') %>% arrange(-sum_value) %>% head(5)

Reporter,Reporter_nm,Flow,sum_weight,sum_value
ABW,ARUBA,Import,419,1801
AGO,ANGOLA,Import,1670777,3249450
ALB,ALBANIA,Export,19885126,13202636
ALB,ALBANIA,Import,14088276,5680646
AND,ANDORRA,Export,7575360,1085887
AND,ANDORRA,Import,106457,10315
ARE,UNITED ARAB EMIRATES,Export,225834713,69352245
ARE,UNITED ARAB EMIRATES,Import,52858666,35525623
ARE,UNITED ARAB EMIRATES,Re-Export,9581072,3980199
ARG,ARGENTINA,Export,153709866,115702253


Reporter,Reporter_nm,Flow,sum_weight,sum_value
CHN,CHINA,Import,54354071286,36336946108
HKG,HONG KONG,Import,24751934145,9466704226
USA,UNITED STATES,Import,3127863369,1909130829
NLD,NETHERLANDS,Import,3501518399,1484676228
DEU,GERMANY,Import,3531762637,1453887673


Reporter,Reporter_nm,Flow,sum_weight,sum_value
USA,UNITED STATES,Export,15525288773,6865813082
HKG,HONG KONG,Export,18268866049,6526781228
JPN,JAPAN,Export,12157602181,5730224769
DEU,GERMANY,Export,11182738775,4824569249
GBR,UNITED KINGDOM,Export,6143680321,2029233554


Reporter,Reporter_nm,Flow,sum_weight,sum_value
PAK,PAKISTAN,Re-Import,10599464,2833013
CAN,CANADA,Re-Import,4545031,2580185
FRA,FRANCE,Re-Import,1856898,1546052
RUS,RUSSIAN FEDERATION,Re-Import,2238450,1150009
PRT,PORTUGAL,Re-Import,969413,785876


Reporter,Reporter_nm,Flow,sum_weight,sum_value
HKG,HONG KONG,Re-Export,15173462459,5598515394
USA,UNITED STATES,Re-Export,142910797,57664328
SAU,SAUDI ARABIA,Re-Export,95103000,27700567
CAN,CANADA,Re-Export,6707501,5563311
ARE,UNITED ARAB EMIRATES,Re-Export,9581072,3980199


### 3.3
- 아래 코드 블록은 대상국이 중국 (ISO 코드가 'CHN'으로 된)인 자료를 대상으로 각각 순중량과 교역액의 연도별 변화추이를 정리하는 내용입니다. 코드의 흐름을 잘 관찰해 보시기 바랍니다.
- 중간에 `dplyr::select`로 표기된 부분은 채워넣을 값을 각각 순중량과 교역액으로 한정하는 역할을 합니다.

In [20]:
pla_wc %>% filter(Partner == 'CHN') %>% group_by(Reporter, Reporter_nm, Flow, year) %>% 
    summarize_at(.vars = vars(Netweight_kg, Value_usd), .funs = funs(sum)) %>% dplyr::select(-Value_usd) %>% spread(key = year, value = Netweight_kg) -> pla_wcw_wide


pla_wc %>% filter(Partner == 'CHN') %>% group_by(Reporter, Reporter_nm, Flow, year) %>% 
    summarize_at(.vars = vars(Netweight_kg, Value_usd), .funs = funs(sum)) %>% dplyr::select(-Netweight_kg) %>% spread(key = year, value = Value_usd) -> pla_wcv_wide

pla_wcw_wide %>% head(12)
pla_wcv_wide %>% head(12)

Reporter,Reporter_nm,Flow,2010,2011,2012,2013,2014,2015,2016,2017
AGO,ANGOLA,Import,103322,8511,803,41553,2468,13927,NA,NA
ALB,ALBANIA,Export,812861,925531,794717,1398855,NA,NA,NA,NA
ALB,ALBANIA,Import,NA,NA,NA,18720,NA,NA,NA,NA
ARE,UNITED ARAB EMIRATES,Export,6955883,11371547,13936582,49877369,15891719,12303725,8617665,NA
ARE,UNITED ARAB EMIRATES,Import,637212,1124450,1864281,2221856,3374501,3157278,2143907,NA
ARE,UNITED ARAB EMIRATES,Re-Export,1721050,134500,714308,134810,313955,106530,NA,NA
ARG,ARGENTINA,Export,25904750,25288490,21129471,14917322,8540762,7674051,3269874,1712664
ARG,ARGENTINA,Import,NA,NA,NA,NA,NA,NA,NA,91610
ARM,ARMENIA,Export,457437,NA,NA,NA,NA,NA,NA,NA
ARM,ARMENIA,Import,NA,NA,NA,NA,2,NA,9450,NA


Reporter,Reporter_nm,Flow,2010,2011,2012,2013,2014,2015,2016,2017
AGO,ANGOLA,Import,104766,7180,378,109724,7200,28216,NA,NA
ALB,ALBANIA,Export,425149,619139,565976,962869,NA,NA,NA,NA
ALB,ALBANIA,Import,NA,NA,NA,13482,NA,NA,NA,NA
ARE,UNITED ARAB EMIRATES,Export,2071036,3991057,4889397,4501610,4985469,4330899,2852756,NA
ARE,UNITED ARAB EMIRATES,Import,694508,575945,1139183,1224180,2123569,2010681,1114514,NA
ARE,UNITED ARAB EMIRATES,Re-Export,413230,29866,293564,40669,103677,68523,NA,NA
ARG,ARGENTINA,Export,17224578,21495661,17552048,10238801,4051357,2991986,1242567,748326
ARG,ARGENTINA,Import,NA,NA,NA,NA,NA,NA,NA,28632
ARM,ARMENIA,Export,353684,NA,NA,NA,NA,NA,NA,NA
ARM,ARMENIA,Import,NA,NA,NA,NA,176,NA,3781,NA


Reporter,Reporter_nm,Flow,2010,2011,2012,2013,2014,2015,2016,2017
KOR,"KOREA, REPUBLIC OF",Export,171802127,141239829,157891729,154110543,146778772,148748722,151366129,119574904
KOR,"KOREA, REPUBLIC OF",Import,3652620,2884578,1538283,1859189,2623824,2379751,842278,1799405


Reporter,Reporter_nm,Flow,2010,2011,2012,2013,2014,2015,2016,2017
KOR,"KOREA, REPUBLIC OF",Export,55567324,50194885,51194379,56344812,47579141,43200394,39844456,25191637
KOR,"KOREA, REPUBLIC OF",Import,2459961,2893720,1128061,1011959,1420037,1026864,440171,2621146


### 3.4
- 3.3에서 연도를 분류 속성으로 한 wide format data frame을 생성했으므로, 시계열 탐색은 해당 data.frame으로부터 `filter` 함수를 적용하여 탐색해 봅니다.
- 아래 결과를 보면, 한국의 대중국 폐플라스틱 무역은 재수출이나 재수입이 없는 수출과 수입만으로 이루어져 있음을 알 수 있고, 수출순중량에 다소 등락이 있는 반면 수출액은 경향적인 감소 추세를 보이고 있음을 알 수 있습니다.

In [21]:
pla_wcw_wide %>% filter(Reporter == 'KOR')
pla_wcv_wide %>% filter(Reporter == 'KOR')

Reporter,Reporter_nm,Flow,2010,2011,2012,2013,2014,2015,2016,2017
KOR,"KOREA, REPUBLIC OF",Export,171802127,141239829,157891729,154110543,146778772,148748722,151366129,119574904
KOR,"KOREA, REPUBLIC OF",Import,3652620,2884578,1538283,1859189,2623824,2379751,842278,1799405


Reporter,Reporter_nm,Flow,2010,2011,2012,2013,2014,2015,2016,2017
KOR,"KOREA, REPUBLIC OF",Export,55567324,50194885,51194379,56344812,47579141,43200394,39844456,25191637
KOR,"KOREA, REPUBLIC OF",Import,2459961,2893720,1128061,1011959,1420037,1026864,440171,2621146


### 3.5
- 3.1에서 만든 표로부터 메트릭 톤당 '수입가격'과 '수출가격'을 계산해 보겠습니다.
- 참고: 수입가격은 물품구입대금에 관세, 운송비, 금융비용 등이 더해지므로 수출가격보다 높은 경향이 있습니다. 수입가격이 높다는 것은, 그 배경에 있는 국내가격이 높거나 가공제품 후 재수출가격이 높음을 뜻합니다.
- 그런데 2017년에 평균 수출가격이 평균 수입가격을 앞서는 현상이 나타납니다. 이는 나머지 7년 간의 패턴과 상반된 것으로, 수입국의 수입유인이 매우 낮아졌음을 의미합니다. 이로부터 4월의 재활용 대란의 배경이 어느 정도 설명됩니다.
- 더 깊은 이야기는 원료인 석유류 및 석유화학제품의 국제무역 자료 분석, 그리고 각국의 플라스틱 재생 업계의 업황 분석을 통해서 이어갈 수 있을 것입니다.

In [24]:
pla_wcyfsum %>% mutate(prunit = 1000 * sum_value / sum_netweight) -> pla_wcyf_price# convert price per metric ton
pla_wcyf_price
pla_wcyf_price %>% dplyr::select(-starts_with('sum_')) %>% spread(key = year, value = prunit)

all,year,Flow,sum_netweight,sum_value,prunit
World,2010,Export,15337934825,6055398026,394.7988
World,2010,Import,15788234286,8067396001,510.9752
World,2010,Re-Export,2715696936,778328090,286.6034
World,2010,Re-Import,1388963,777764,559.9602
World,2011,Export,14822716997,7024457937,473.8981
World,2011,Import,15974271606,9618215653,602.1067
World,2011,Re-Export,2655678460,860032180,323.8465
World,2011,Re-Import,1748614,1257839,719.3349
World,2012,Export,14841538079,6891275058,464.3235
World,2012,Import,15940073245,9546979599,598.9295


all,Flow,2010,2011,2012,2013,2014,2015,2016,2017
World,Export,394.7988,473.8981,464.3235,508.5623,440.7112,394.8607,375.8203,428.0451
World,Import,510.9752,602.1067,598.9295,630.3514,602.1517,475.2025,424.0977,384.4751
World,Re-Export,286.6034,323.8465,326.5992,551.1614,308.4003,319.8266,502.4397,44122.7974
World,Re-Import,559.9602,719.3349,749.8858,489.7788,661.0934,357.3360,326.7848,388.6555


In [28]:
pla_wc %>% filter(Reporter == 'KOR') %>% group_by(Partner, Partner_nm, Flow, year) %>% 
    summarize_at(.vars = vars(Netweight_kg, Value_usd), .funs = funs(sum)) %>% dplyr::select(-Value_usd) %>% 
    spread(key = year, value = Netweight_kg) -> pla_wkw_wide
pla_wc %>% filter(Reporter == 'KOR') %>% group_by(Partner, Partner_nm, Flow, year) %>% 
    summarize_at(.vars = vars(Netweight_kg, Value_usd), .funs = funs(sum)) %>% dplyr::select(-Netweight_kg) %>% 
    spread(key = year, value = Value_usd) -> pla_wkv_wide

pla_wkw_wide %>% arrange(-`2017`) %>% head(5)
pla_wkv_wide %>% arrange(-`2017`) %>% head(5)

Partner,Partner_nm,Flow,2010,2011,2012,2013,2014,2015,2016,2017
CHN,CHINA,Export,171802127,141239829,157891729,154110543,146778772,148748722,151366129,119574904
VNM,VIET NAM,Export,7632766,4097892,1811658,2483151,8417985,11321717,21546755,50787060
JPN,JAPAN,Import,32691295,41433291,45530659,43446002,39880861,35897651,29418449,31117971
MYS,MALAYSIA,Export,6318334,6134217,6906852,7617611,7679957,11664292,11682327,12984899
PHL,PHILIPPINES,Import,203598,1682466,2796664,2022117,4244184,3872531,3649437,10544768


Partner,Partner_nm,Flow,2010,2011,2012,2013,2014,2015,2016,2017
CHN,CHINA,Export,55567324,50194885,51194379,56344812,47579141,43200394,39844456,25191637
VNM,VIET NAM,Export,1303549,710004,720211,723572,2022539,3334737,4369061,11628842
JPN,JAPAN,Import,14559547,19690602,20268919,16334759,14736685,11528181,9515490,10058088
MYS,MALAYSIA,Export,1827208,1886641,2276696,2843514,3344124,4263232,3876337,4320140
VNM,VIET NAM,Import,337040,703977,925765,918721,1326669,1741148,2893032,4036748


## Assignment
### Preprocessing (no presentation)
+ 제공된 2015년 세계 무역 데이터를 이용하십시오.
    + CSV 파일을 불러오십시오.
    + 값의 형식이 알맞게 불러들여졌는지 확인하십시오. 만약 알맞지 않다면, 적절한 방식으로 변환하십시오. 오류가 없다면 넘어가도 됩니다.
    + 필드 이름에 빈칸이 포함되어 이용하기 불편하므로, 각자가 원하는 방식으로 필드 이름을 새롭게 지정하십시오.
        + 문제에서 이야기했듯이 필드 이름에는 빈칸이 포함되지 않아야 합니다.
        + 만약 빈칸이 들어간 필드를 이용해서 SAC 관련 함수들을 이용하려면, 필드 이름을 \`\` 사이에 넣어야 합니다. (주의: 작은 따옴표 아님! 1 왼쪽의 물결무늬 표시 (~)를 Shift 없이 타이핑할 때 나오는 기호입니다) 
        + 필드 이름은 가급적 숫자로 시작하지 않도록 지정하십시오. 숫자로 시작한 필드도 \`\` 사이에 넣어 처리해야 합니다.
    + `unique` 함수와 파이프 연산자를 이용해서 한국 (ISO code: KOR)의 수입 품목이 몇 가지인지 확인하십시오.
    + 다른 국가의 현황도 살펴보십시오.

### Presentation
+ 본인이 원하는 품목을 한 가지 지정한 후, 위에서 보인 폐플라스틱 예시와 동일한 요령으로 분석하여 5분 이내로 발표하십시오. 강조하여야 할 점은 아래와 같습니다.
    + 지정품목의 전세계 교역량: 중량과 교역액을 분리하여
    + Top X 교역대상국 (수출, 수입 총합): 중량과 교역액을 분리하여
    + Top X 교역대상국 (수출, 수입 분리): 중량과 교역액을 분리하여